**Loading the libraries of pandas, numpy, re, sklearn models and count vectorizer.**

In [2]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import StratifiedKFold

**Determining the number of folds, set to 3**

In [3]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=3)

**Reading the dataset in dataframe, and naming the columns**

In [4]:
df=pd.read_table('https://raw.githubusercontent.com/justmarkham/pycon-2016-tutorial/master/data/sms.tsv', header = None)
df.columns = ['Category','Text']

**Defining a function to convert the given sentences into a sparse matrix, and return it as a dataframe**

In [5]:
def vect(raw_x_train):
    vectorizer = CountVectorizer( )
    X = vectorizer.fit_transform(raw_x_train.tolist())
    return(pd.DataFrame(X.toarray(),index = raw_x_train.index, columns = vectorizer.get_feature_names()), vectorizer)

**Defining a function to calculate the required probabilities**

In [6]:
def dictionary(x_train, y_train):
    dict1 = (y_train.groupby(y_train).size()/y_train.shape[0]).to_dict() #Calculating probabilties of Each Test Class Occurrence
    didict = {}   #Initializing the Dictionary
    for i in dict1:   #For each class
        denom = (  (x_train.loc[(y_train[y_train == i].index)].values.sum()) + (len(x_train.columns))   ) #Calculating the denominator for NB. Added the length too for Laplace Smoothing
        didict[i] = (((x_train.loc[(y_train[y_train == i].index)]).sum()) + 1  )/denom  #Under each test class, storing the independent probability for each word or train column given the considered test class 
    return(dict1, didict)  # returning both these classes

**Defining a function to predict the values/ probabilities**

In [8]:
def predict_val(x_test, dict1, didict):
    predict_list1 = np.empty((0,2))   # Initializing a np array to store prediction values
    for sentence1 in x_test:          #For every sentence(value) in the test dataset
        list1 = []                    #Initializing a list to store values of each sentence
        for classes in dict1:         #For every class in our dictionary
            val = 1                   #Initializing a value with 1 so as it gets mutiplied 
            for word in re.findall('\w+',sentence1):              #For every word in the sentence(value)
                try:
                    val = val * didict[classes][word.lower()]     #Lowercasing the word, then multilying the respective probability for that specific class
                except Exception:
                    pass                                          #Passing in case, the word is not found in our training set
            (list1.append(val*dict1[classes]))                    #Storing both(one of each class) the probabilities in the list 
            #print(len(list1))
        predict_list1 = np.append(predict_list1,np.array([list1]), axis = 0)   #Storing the above lists (one for each sentence) in our np array
    return(predict_list1)

**Defining a function to translate the values to class labels**

In [9]:
def predict_class(predict_list1, dict1):
    list2 = []
    for i in range(predict_list1.shape[0]):
        list2.append(list(dict1)[np.where(predict_list1[i]==np.max(predict_list1[i]))[0][0]]) #Returns the index of the maximum value in each row, substitutes in dictionary, and returns the name of the class
    return(np.array(list2))

**A function that brings together all the functions above**

In [10]:
def nlp(df):
    x = df['Text']
    y = df['Category']
    final_predictions = pd.Series(index = df.index)
    for train,test in skf.split(x,y):
        x_train = x[train]
        y_train = y[train]
        x_train_df, vectorizer = vect(x_train)
        class_dict, model_dict = dictionary(x_train_df, y_train)
        print(class_dict)
        x_test = x[test]
        y_test = y[test]
        predict_probs = predict_val(x_test, class_dict , model_dict)
        predictions = predict_class(predict_probs, class_dict)
        final_predictions.loc[test] = predictions
        accuracy = (predictions == y_test).sum()/len(y_test)
    final_acc = (final_predictions == y).sum()/ len(y)
    print(final_acc)
    return(final_predictions)

In [11]:
new_df = nlp(df)

C:\conda\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


{'ham': 0.8659127625201939, 'spam': 0.13408723747980614}
{'ham': 0.8659488559892329, 'spam': 0.13405114401076715}
{'ham': 0.8659488559892329, 'spam': 0.13405114401076715}
0.9860014357501795
